# Tutorial 4 - Defining a custom ```HilbertSpace```

Within this tutorial we show through a simple but useful example how a user can define their own ```HilbertSpace```. A key point is that instances of ```HilbertSpace``` should not, in general, be constructed directly. Rather the intention is for the user to define a new class derived from ```HilbertSpace```. 

We have already seen that ```pygeoinf``` has an implementation of Euclidean space. Here, we will generalise this to allow for a non-standard inner product. Elements of our space, $X$, will be vectors in $\mathbb{R}^{n}$, but we use the inner product
$$
(x, y)_{X} = (Mx, y)_{\mathbb{R}^{n}}, 
$$
with $M$ an symmetric and positive-definite matrix that we call the **metric**. From this definition, it follows that the Riesz mapping, $J_{X}$ on a component-level is just multiplication by $M$. 

The implementation of the necessary class is given below. 

In [6]:
import numpy as np
from scipy.linalg import cho_factor, cho_solve
from pygeoinf import  HilbertSpace, LinearForm

class GeneralisedEuclideanSpace(HilbertSpace):
    """
    Class for a generalised Euclidean space whose inner
    product is defined by a metrix tensor. 
    """

    def __init__(self, dim, metric, /, *, inverse_metric=None):
        """
        Args:
            dim (int): Dimension of the space. 
            metric (matrix-like): The metric tensor. 
        """        
        self._metric = metric
        self._factor = cho_factor(metric)                        
        super().__init__(dim, self._to_components, self._from_components, self._inner_product, self._to_dual, self._from_dual)

    def _to_components(self, x):
        return x

    def _from_components(self, c):
        return c

    def _inner_product(self, x1, x2):
        return np.dot(self._metric @ x1, x2)    

    def _from_dual(self, xp):
        cp = self.dual.to_components(xp)
        c = cho_solve(self._factor, cp)        
        return self.from_components(c)

    def _to_dual(self, x):
        cp = self._metric @ x
        return self.dual.from_components(cp)

Before discussing how this class has been defined, let's see it in action:

In [7]:
# Set the dimension of the space. 
dim = 4

# Generate a metric tensor as a self-adjoint matrix. 
metric = np.random.randn(dim, dim)
metric = np.identity(dim) + 5 * metric @ metric.T

print(f'The metric is:\n {metric}')

# Construct the Hilbert space. 
X = GeneralisedEuclideanSpace(dim, metric)

# Test the basic identities on random vectors. 
x = X.random()
y = X.random()

print(f'x      = {x}')
print(f'y      = {y}')
print(f'(x,y)  = {X.inner_product(x,y)}')

xp = X.to_dual(x)
print(f'xp     = {xp}')
print(f'<xp,y> = {xp(y)}')

zp = X.dual.random()
print(f'zp     = {zp}')
print(f'<zp,y> = {zp(y)}')
z = X.from_dual(zp)
print(f'z      = {z}')
print(f'(z,y)  = {X.inner_product(z,y)}')



The metric is:
 [[ 6.76766443 -4.57721065  0.09136889 -3.52186613]
 [-4.57721065 10.27548365  3.54982226  6.54660035]
 [ 0.09136889  3.54982226 10.98445619  2.77562685]
 [-3.52186613  6.54660035  2.77562685  5.6913453 ]]
x      = [ 0.41947337 -0.25107355 -0.79619042 -1.23571486]
y      = [-0.23816025 -0.20301065  0.57807871 -0.59881688]
(x,y)  = 1.0327941304767894
xp     = [  8.26734679 -15.41598594 -13.02854183 -12.36381471]
<xp,y> = 1.0327941304767894
zp     = [ 0.82876711 -0.36423126 -0.65252455  1.33937719]
<zp,y> = -1.3026887782920733
z      = [ 0.3640404  -0.60051484 -0.18169014  1.23997281]
(z,y)  = -1.3026887782920729


### The constructor

The ```__init__``` method for the class takes in two arguments:

- The dimension of the space
- The metric tensor, $M$, assumed to be a numpy matrix. 

The Cholesky factorisation of the matrix is computed and stored. This allows for efficient solution of the linear equation $y = Mx$ as required within the Riesz mapping. The base class is then initialised, with implementations for the necessary mappings provided through private methods. As a general comment, these methods make use of properties derived from the base class. But there is no issue of circularity because these methods cannot be called until the base class has been constructed.

The implementation could be improved in various ways. We might, for example, allow the metric to be given in a more general form (e.g., as a ```scipy.sparse``` ```LinearOperator```). We might also allow the user to optionally provide the inverse metric directly. Such steps would be particularly useful if the dimension of the space is large. 

### To and from component mappings

These are just the identity mappings. This is identical to what is done in ```EuclideanSpace```. 

### Inner product

The metric is used along with the standard  ```numpy``` inner product. In ```EuclideanSpace``` the only difference is that the metric is the identity, and hence the matrix-vector product it not needed. 

### To and from dual mappings. 

This is the only point at which this implementation of this space differs significantly from that for ```EuclideanSpace```. To map a vector to the dual space, we need to form its components, multiply by the metric, and then form the corresponding dual vector. In ```EuclideanSpace``` the metric is the identity and so the middle step is skipped. For the inverse mapping the process is analogous, but now we need to multiply by the inverse metric in the middle step. Within our implementation this is done efficiently using back-substitution having stored the Choelsky factorisation of the metric. 